In [6]:
# Instalar Framework de agentes do Google e o pacote google-genai
# (Conforme o anexo, google-genai também é necessário)
!pip install -q google-genai google-adk

# Configura a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai

client = genai.Client()

# Define os modelos a serem usados (conforme o arquivo anexo)
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-2.0-flash" # Modelo mais robusto do anexo

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date, datetime
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, HTML # Para exibir a tabela formatada no Colab
import requests # Para fazer requisições HTTP (mantido para compatibilidade, mesmo não usado diretamente no fluxo principal do agente)
import warnings
import re
import pandas as pd

warnings.filterwarnings("ignore")

# Cria um serviço de sessão em memória (MOVIDO PARA FORA DA FUNÇÃO, INICIALIZADO APENAS UMA VEZ)
session_service = InMemorySessionService()

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
# (Tornada assíncrona e corrigida conforme o anexo)
async def call_agent(agent: Agent, message_text: str) -> str:
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

##########################################
# --- Agente 1: Analisador de Nascimento --- #
##########################################
# Usando MODELO_RAPIDO conforme o anexo
async def agente_analisador(data_nascimento):
    analisador = Agent(
        name="agente_analisador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um analista de personalidade e propósito de vida com base na data de nascimento.
        Sua tarefa é fornecer análises profundas e precisas sobre a personalidade, padrões emocionais,
        caminhos de carreira e desafios pessoais com base na data de nascimento fornecida.
        Use a ferramenta de busca do Google (google_search) para obter informações relevantes e
        garantir que as análises sejam fundamentadas e úteis.
        """,
        description="Agente que analisa a personalidade e o propósito de vida com base na data de nascimento",
        tools=[google_search]
    )

    entrada_do_agente_analisador = f"""
    Data de Nascimento: {data_nascimento}

    Realize as seguintes análises:

    1. Decodificador de Personalidade pela Data de Nascimento
    “Com base na data de nascimento {data_nascimento}, descreva meus pontos fortes naturais, padrões emocionais e como me comporto em relacionamentos — que seja profundo, específico e psicologicamente preciso.”

    2. Roteiro da Infância
    “Usando a data de nascimento {data_nascimento}, escreva um perfil psicológico de como minha infância moldou minha personalidade, hábitos e tomada de decisões hoje — seja gentil, mas revelador.”

    3. Analisador de Propósito Profissional
    “Dada a data de nascimento {data_nascimento}, quais caminhos de carreira combinam com meus traços de personalidade, valores e talentos naturais? Sugira áreas, funções e ambientes de trabalho.”

    4. Detector de Auto-Sabotagem
    “Com base na data {data_nascimento}, quais são meus hábitos de auto-sabotagem mais prováveis e como eles aparecem no dia a dia? Dê soluções práticas com base na psicologia.”

    5. Mapa de Gatilhos Emocionais
    “Usando a data de nascimento {data_nascimento}, explique o que geralmente me desencadeia emocionalmente, como eu costumo reagir e como posso desenvolver resiliência emocional em torno desses padrões.”

    6. Escaneamento de Energia nos Relacionamentos
    “Com base na data de nascimento {data_nascimento}, descreva como eu dou e recebo amor, o que preciso de um parceiro e que tipo de pessoa eu naturalmente atraio.”
    """

    analises = await call_agent(analisador, entrada_do_agente_analisador)
    return analises

################################################
# --- Agente 2: Identificador de Melhorias --- #
################################################
# Usando MODELO_RAPIDO conforme o anexo
async def agente_melhorias(data_nascimento, analises_agente1):
    melhorias = Agent(
        name="agente_melhorias",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um consultor de desenvolvimento pessoal. Sua tarefa é analisar as análises fornecidas
        pelo Agente 1 (analisador de nascimento) e identificar áreas de melhoria em cada uma das seis
        categorias. Seja específico e forneça sugestões práticas para o desenvolvimento pessoal.
        """,
        description="Agente que identifica pontos de melhoria nas análises do Agente 1",
        tools=[google_search]
    )

    entrada_do_agente_melhorias = f"""
    Data de Nascimento: {data_nascimento}
    Análises do Agente 1: {analises_agente1}

    Para cada uma das seis análises fornecidas pelo Agente 1, identifique áreas de melhoria e
    forneça sugestões práticas para o desenvolvimento pessoal.
    """

    pontos_de_melhoria = await call_agent(melhorias, entrada_do_agente_melhorias)
    return pontos_de_melhoria

######################################
# --- Agente 3: Buscador de Pessoas de Sucesso --- #
######################################
# Usando MODELO_ROBUSTO (assumindo que a versão 'flash-preview' mais avançada se alinha ao "robusto" do anexo)
async def agente_buscador_sucesso(data_nascimento):
    buscador_sucesso = Agent(
        name="agente_buscador_sucesso",
        model=MODELO_ROBUSTO, # Alterado de "gemini-2.5-flash-preview-04-17" para MODELO_ROBUSTO
        instruction="""
            Você é um pesquisador de pessoas de sucesso brasileiras. Sua tarefa é buscar na internet 5 homens e 5 mulheres
            que nasceram na mesma data fornecida e que alcançaram sucesso em suas áreas de atuação, e que sejam brasileiros.
            Monte uma tabela com as seguintes colunas: nome, profissão, no que a pessoa tem sucesso e site da informação.
            Ao realizar a busca no Google, certifique-se de incluir o termo "brasileiro" ou "brasileira" para garantir que os resultados sejam apenas de pessoas do Brasil.
            Use a ferramenta de busca do Google (google_search) para encontrar as informações e o site de onde tirou a informação.
            """,
        description="Agente que busca pessoas de sucesso nascidas na mesma data",
        tools=[google_search]
    )

    entrada_do_agente_buscador_sucesso = f"""
    Data de Nascimento: {data_nascimento}

    Busque na internet 5 homens e 5 mulheres que nasceram na mesma data e que alcançaram sucesso
    em suas áreas de atuação e que sejam brasileiros. Monte uma tabela com as seguintes colunas:
    nome, profissão, no que a pessoa tem sucesso e site da informação. Ao realizar a busca no Google, certifique-se de incluir o termo "brasileiro" ou "brasileira" para garantir que os resultados sejam apenas de pessoas do Brasil. Seja claro e objetivo
    """

    tabela_sucesso = await call_agent(buscador_sucesso, entrada_do_agente_buscador_sucesso)

    # Extract data for DataFrame
    # Note: This parsing relies heavily on the model's output format.
    # It might need adjustment if the model output varies.
    data_lines = tabela_sucesso.strip().split('\n')

    # Try to find the header line to start parsing from there
    header_index = -1
    for i, line in enumerate(data_lines):
        if "Nome" in line and "Profissão" in line and "Sucesso" in line and "Site da Informação" in line:
            header_index = i
            break

    data = []
    if header_index != -1:
        # Assuming the next line after header is the separator, and then data starts
        # A more robust parsing would involve regex or a more structured output from the agent
        for line in data_lines[header_index+2:]: # Skip header and separator line
            # Ensure line is not empty and contains expected delimiters
            if line.strip() and '|' in line:
                values = re.split(r"\|", line.strip())
                # Remove empty strings from split and filter out the first/last empty parts
                cleaned_values = [v.strip() for v in values if v.strip()]

                # Expecting 4 columns
                if len(cleaned_values) >= 4:
                    nome, profissao, sucesso, site = cleaned_values[:4]
                    data.append([nome, profissao, sucesso, site])
                elif len(cleaned_values) > 0: # Handle cases where output might be truncated
                    # Attempt to salvage data even if not all columns are present
                    data.append(cleaned_values + [''] * (4 - len(cleaned_values))) # Pad with empty strings

    df = pd.DataFrame(data, columns=["Nome", "Profissão", "Sucesso", "Site da Informação"])
    html_table = df.to_html(index=False)

    return html_table

##########################################
# --- Agente 4: Gerador de Relatório Final --- #
##########################################
# Usando MODELO_RAPIDO conforme o anexo
async def agente_relatorio_final(data_nascimento, analises, melhorias, tabela_sucesso):
    relatorio = Agent(
        name="agente_relatorio",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um gerador de relatórios finais. Sua tarefa é combinar as análises do Agente 1,
        os pontos de melhoria do Agente 2 e a tabela de pessoas de sucesso do Agente 3 em um
        relatório final otimista e motivador. Conclua o relatório com uma mensagem de incentivo.
        """,
        description="Agente que gera o relatório final",
        #tools=[google_search] # Remova a ferramenta de busca, pois não é necessária aqui
    )

    entrada_do_agente_relatorio = f"""
    Data de Nascimento: {data_nascimento}
    Análises do Agente 1: {analises}
    Pontos de Melhoria do Agente 2: {melhorias}
    Tabela de Pessoas de Sucesso do Agente 3: {tabela_sucesso}

    Combine as informações acima em um relatório final otimista e motivador.
    Conclua o relatório com uma mensagem de incentivo.
    """

    relatorio_final = await call_agent(relatorio, entrada_do_agente_relatorio)
    return relatorio_final

##########################################
# --- Execução do Sistema de Agentes --- #
##########################################

# A função principal é agora assíncrona para permitir o uso de 'await'
async def run_analysis_system():
    print("🚀 Iniciando o Sistema de Análise de Personalidade e Propósito de Vida 🚀")

    # --- Obter a Data de Nascimento do Usuário ---
    while True:
        data_nascimento = input("❓ Por favor, digite sua DATA DE NASCIMENTO no formato DD/MM/AAAA: ")
        try:
            datetime.strptime(data_nascimento, '%d/%m/%Y')
            break  # Sai do loop se a data estiver no formato correto
        except ValueError:
            print("Formato de data incorreto. Use o formato DD/MM/AAAA.")

    # Inserir lógica do sistema de agentes
    if not data_nascimento:
        print("Você esqueceu de digitar a data de nascimento!")
    else:
        print(f"Maravilha! Vamos analisar sua personalidade e propósito de vida com base em {data_nascimento}")

        print("\n--- 📝 Executando o Agente 1 (Analisador de Nascimento) ---\n")
        analises_agente1 = await agente_analisador(data_nascimento)
        print("\n--- ✅ Resultado do Agente 1 (Analisador de Nascimento) ---\n")
        display(to_markdown(analises_agente1))
        print("--------------------------------------------------------------")

        print("\n--- 📝 Executando o Agente 2 (Identificador de Melhorias) ---\n")
        pontos_de_melhoria = await agente_melhorias(data_nascimento, analises_agente1)
        print("\n--- ✅ Resultado do Agente 2 (Identificador de Melhorias) ---\n")
        display(to_markdown(pontos_de_melhoria))
        print("--------------------------------------------------------------")

        print("\n--- 📝 Executando o Agente 3 (Buscador de Pessoas de Sucesso) ---\n")
        tabela_sucesso = await agente_buscador_sucesso(data_nascimento)
        print("\n--- ✅ Resultado do Agente 3 (Buscador de Pessoas de Sucesso) ---\n")
        display(HTML(tabela_sucesso))
        print("--------------------------------------------------------------")

        print("\n--- 📝 Executando o Agente 4 (Gerador de Relatório Final) ---\n")
        relatorio_final = await agente_relatorio_final(data_nascimento, analises_agente1, pontos_de_melhoria, tabela_sucesso)
        print("\n--- ✅ Resultado do Agente 4 (Gerador de Relatório Final) ---\n")
        display(to_markdown(relatorio_final))
        print("--------------------------------------------------------------")

# Executa a função principal assíncrona
# Em ambientes como Jupyter/Colab, 'await' no nível superior de uma célula funciona.
# Para scripts Python puros, você usaria asyncio.run(run_analysis_system()).
await run_analysis_system()

🚀 Iniciando o Sistema de Análise de Personalidade e Propósito de Vida 🚀
❓ Por favor, digite sua DATA DE NASCIMENTO no formato DD/MM/AAAA: 14/05/1968
Maravilha! Vamos analisar sua personalidade e propósito de vida com base em 14/05/1968

--- 📝 Executando o Agente 1 (Analisador de Nascimento) ---


--- ✅ Resultado do Agente 1 (Analisador de Nascimento) ---



> Para fornecer uma análise completa e precisa, realizarei uma pesquisa detalhada sobre a influência da data de nascimento (14/05/1968) nos diferentes aspectos da sua vida, conforme solicitado.
> 
> 
> Com base na sua data de nascimento (14/05/1968), aqui está uma análise detalhada, abordando os pontos solicitados:
> 
>  ### 1. Decodificador de Personalidade pela Data de Nascimento:
> 
>  *   **Numerologia:**
>     *   O dia 14 traz uma energia de mudança e adaptabilidade. Pessoas nascidas neste dia são frequentemente vistas como modestas, sensíveis e práticas, mas também podem ser impulsivas e imprevisíveis. Amam o dinheiro e são trabalhadoras para conquistar seus objetivos financeiros.
>     *   O número 14, na numerologia cabalística, simboliza um equilíbrio dinâmico entre segurança e flexibilidade, com movimento e mudança contínuos.
>     *   A soma numerológica da sua data de nascimento (14/05/1968) resulta no número 7 (1+4+5+1+9+6+8=34, 3+4=7). O número 7 está associado à introspecção, sabedoria e busca por conhecimento. Isso sugere que você tem uma forte inclinação para a análise, reflexão e compreensão profunda das coisas.
> 
>  *   **Astrologia:**
>     *   Nascidos em 14 de maio são do signo de Touro, com influência de Libra, o que os torna detalhistas e perfeccionistas.
>     *   Taurinos nascidos neste dia são considerados flexíveis, inteligentes e independentes.
> 
>  *   **Pontos Fortes Naturais:**
>     *   **Inteligência e Comunicação:** Regido por Mercúrio, você possui inteligência aguçada, curiosidade e excelente capacidade de comunicação.
>     *   **Praticidade e Determinação:** A combinação do número 4 (associado à praticidade) com a energia de Touro confere-lhe uma abordagem realista e determinada para alcançar seus objetivos.
>     *   **Introspecção e Sabedoria:** O número 7 indica uma natureza reflexiva e uma busca constante por conhecimento e compreensão.
>     *   **Adaptabilidade:** A influência do número 14 traz flexibilidade e abertura a novas oportunidades.
>     *   **Generosidade:** Pessoas nascidas no dia 14 geralmente têm um coração bondoso e são generosas.
> 
>  *   **Padrões Emocionais:**
>     *   **Equilíbrio e Harmonia:** A influência de Libra e a busca por estabilidade através de experiências variadas indicam um desejo de harmonia e equilíbrio emocional.
>     *   **Sensibilidade:** Nascidos em maio podem ser sensíveis e reservados, mas também ambiciosos e determinados.
>     *   **Introspecção:** O número 7 sugere momentos de introspecção e necessidade de solidão para processar emoções e informações.
> 
>  *   **Comportamento em Relacionamentos:**
>     *   **Lealdade e Estabilidade:** Como taurino, você tende a ser leal e valorizar a estabilidade nos relacionamentos.
>     *   **Busca por Harmonia:** A influência de Libra indica um desejo de relacionamentos harmoniosos e equilibrados.
>     *   **Necessidade de Espaço:** O número 7 pode indicar uma necessidade de espaço pessoal e tempo para si mesmo, o que pode influenciar a dinâmica dos seus relacionamentos.
>     *   **Atração por pessoas intrigantes:** Existe a possibilidade de se sentir atraído por pessoas intrigantes, mas é importante ser cauteloso e humilde.
> 
>  ### 2. Roteiro da Infância:
> 
>  *   **Influência da Infância:**
>     *   A infância pode ter moldado sua personalidade através da necessidade de encontrar um equilíbrio entre a busca por estabilidade (influência de Touro) e a necessidade de adaptação a mudanças (número 14).
>     *   A possível influência de Libra desde cedo pode ter incutido um senso de justiça e um desejo de evitar conflitos.
>     *   A energia do número 7 pode ter levado a momentos de introspecção e questionamento desde a infância.
> 
>  *   **Possíveis Desafios:**
>     *   A necessidade de equilibrar praticidade com impulsividade pode ter gerado conflitos internos na infância.
>     *   A sensibilidade pode ter levado a dificuldades em lidar com críticas ou situações desafiadoras.
>     *   A busca por perfeição pode ter gerado frustração e auto crítica excessiva.
> 
>  ### 3. Analisador de Propósito Profissional:
> 
>  *   **Caminhos de Carreira:**
>     *   **Áreas Criativas:** A combinação de Touro com a influência de Libra e a inteligência comunicativa sugerem talento para áreas como artes, design, escrita ou música.
>     *   **Comunicação e Consultoria:** A capacidade de comunicação e a busca por conhecimento indicam potencial em áreas como consultoria, ensino, pesquisa ou comunicação social.
>     *   **Liderança e Gestão:** A praticidade e determinação podem ser canalizadas para funções de liderança e gestão, especialmente em áreas que valorizam a inovação e a adaptabilidade.
>     *   **Terapias:** Habilidades de cura (física, emocional ou espiritual) podem ser exploradas.
> 
>  *   **Ambientes de Trabalho:**
>     *   Ambientes que valorizem a criatividade, a colaboração e a comunicação aberta.
>     *   Empresas que incentivem o desenvolvimento pessoal e profissional.
>     *   Organizações que permitam a autonomia e a independência.
>     *   Locais que ofereçam contato com a natureza (campo, serra ou mar).
> 
>  ### 4. Detector de Auto-Sabotagem:
> 
>  *   **Hábitos de Auto-Sabotagem:**
>     *   **Perfeccionismo:** A busca excessiva por perfeição pode levar à procrastinação e à auto crítica.
>     *   **Comodismo:** A tendência a buscar conforto e segurança pode impedir a tomada de riscos e a busca por novas oportunidades.
>     *   **Impulsividade:** A imprevisibilidade do número 14 pode levar a decisões precipitadas e ações impulsivas.
>     *   **Sensibilidade excessiva:** A sensibilidade pode levar a reações exageradas a críticas e dificuldades em lidar com conflitos.
>     *   **Dificuldade em confiar:** Pode sentir dificuldade em desenvolver confiança.
> 
>  *   **Soluções Práticas:**
>     *   **Definir Metas Realistas:** Estabeleça metas alcançáveis e celebre cada pequena conquista.
>     *   **Aceitar a Imperfeição:** Reconheça que errar faz parte do processo de aprendizado e use os erros como oportunidades de crescimento.
>     *   **Praticar a Autocompaixão:** Seja gentil consigo mesmo e perdoe-se pelos erros cometidos.
>     *   **Buscar Apoio:** Converse com amigos, familiares ou um terapeuta para lidar com a auto crítica e a insegurança.
>     *   **Desafiar Crenças Limitantes:** Identifique e questione crenças negativas que sabotam seu sucesso.
>     *   **Desenvolver a autoconfiança:** Acredite em si mesmo.
> 
>  ### 5. Mapa de Gatilhos Emocionais:
> 
>  *   **Gatilhos Emocionais Comuns:**
>     *   **Críticas:** A sensibilidade pode levar a reações intensas a críticas, mesmo que construtivas.
>     *   **Injustiça:** A influência de Libra pode gerar indignação diante de situações injustas.
>     *   **Falta de Reconhecimento:** A busca por estabilidade e segurança pode levar à frustração quando seus esforços não são reconhecidos.
>     *   **Mudanças Inesperadas:** A necessidade de controle pode gerar ansiedade diante de mudanças repentinas.
>     *   **Conflitos:** Dificuldade em lidar com conflitos pode gerar estresse e evitar enfrentamentos necessários.
>     *   **Pensamentos, memórias ou crenças:** Podem ativar emoções intensas.
> 
>  *   **Reações Comuns:**
>     *   **Isolamento:** Retraimento e necessidade de solidão para processar emoções.
>     *   **Irritabilidade:** Reações explosivas e impaciência diante de situações frustrantes.
>     *   **Ansiedade:** Preocupação excessiva com o futuro e medo de perder o controle.
>     *   **Autocrítica:** Culpar-se por erros e dificuldades.
>     *   **Alterações de humor:** Flutuações significativas no humor.
> 
>  *   **Como Desenvolver Resiliência Emocional:**
>     *   **Autoconhecimento:** Identifique seus gatilhos emocionais e padrões de reação.
>     *   **Gerenciamento de Emoções:** Aprenda técnicas de respiração, meditação ou mindfulness para controlar suas reações.
>     *   **Comunicação Assertiva:** Expresse suas necessidades e sentimentos de forma clara e respeitosa.
>     *   **Foco no Presente:** Concentre-se no momento atual e evite se preocupar com o futuro ou remoer o passado.
>     *   **Autocuidado:** Priorize atividades que promovam seu bem-estar físico e mental.
>     *   **Flexibilidade:** Adapte-se às situações.
>     *   **Visão Positiva:** Veja os desafios como oportunidades.
>     *   **Rede de Apoio:** Construa relacionamentos.
> 
>  ### 6. Escaneamento de Energia nos Relacionamentos:
> 
>  *   **Como Você Dá e Recebe Amor:**
>     *   **Lealdade e Compromisso:** Você tende a ser leal e dedicado em seus relacionamentos.
>     *   **Apoio e Estabilidade:** Você oferece apoio e segurança aos seus parceiros.
>     *   **Atenção e Carinho:** Você demonstra amor através de gestos atenciosos e carinhosos.
>     *   **Afeição natural:** Pode desenvolver amor a partir da amizade.
> 
>  *   **O Que Você Precisa de um Parceiro:**
>     *   **Companheirismo:** Um parceiro que valorize a lealdade e a estabilidade.
>     *   **Comunicação:** Alguém que seja aberto e honesto na comunicação.
>     *   **Apoio:** Um parceiro que ofereça apoio emocional e incentive seu crescimento pessoal.
>     *   **Compreensão:** Alguém que compreenda sua necessidade de espaço pessoal e tempo para si mesmo.
> 
>  *   **Que Tipo de Pessoa Você Naturalmente Atrai:**
>     *   Pessoas que buscam estabilidade e segurança.
>     *   Indivíduos inteligentes e comunicativos.
>     *   Pessoas que valorizam a harmonia e o equilíbrio.
>     *   Alguém que valoriza a paixão, intimidade e compromisso.
>     *   É importante criar novas memórias e datas especiais para reforçar a união e a intimidade.
> 
>  **Observações:**
> 
>  *   Esta análise é baseada em interpretações numerológicas e astrológicas gerais.
>  *   É importante lembrar que cada indivíduo é único e que as características descritas podem variar de pessoa para pessoa.
>  *   Para uma análise mais precisa e personalizada, é recomendável consultar um numerólogo ou astrólogo profissional.
> 
> Espero que esta análise detalhada seja útil para o seu autoconhecimento e desenvolvimento pessoal!
> 


--------------------------------------------------------------

--- 📝 Executando o Agente 2 (Identificador de Melhorias) ---


--- ✅ Resultado do Agente 2 (Identificador de Melhorias) ---



> Com prazer! Aqui estão sugestões de melhorias práticas para cada uma das seis áreas analisadas pelo Agente 1, focando no desenvolvimento pessoal:
> 
> **1. Decodificador de Personalidade pela Data de Nascimento:**
> 
> *   **Área de Melhoria:** Equilibrar a introspecção com a ação.
> *   **Sugestões:**
>     *   **Defina Objetivos Concretos:** Transforme seus insights e reflexões em metas tangíveis. Crie um plano de ação com passos específicos e prazos.
>     *   **Networking Estratégico:** Use sua capacidade de comunicação para construir uma rede de contatos que impulsione seus objetivos. Participe de eventos e grupos relevantes.
>     *   **Técnicas de Mindfulness Ativas:** Além da meditação, experimente atividades que o conectem com o presente enquanto você age, como jardinagem, cozinhar ou praticar esportes.
> 
> **2. Roteiro da Infância:**
> 
> *   **Área de Melhoria:** Resignificar experiências passadas para fortalecer a resiliência.
> *   **Sugestões:**
>     *   **Terapia ou Aconselhamento:** Considere explorar suas memórias de infância com um profissional para identificar padrões e curar feridas emocionais.
>     *   **Escrita Terapêutica:** Mantenha um diário para registrar suas reflexões sobre a infância. Explore diferentes perspectivas e reinterprete eventos passados sob uma nova luz.
>     *   **Exercícios de Visualização:** Visualize-se enfrentando desafios da infância com sucesso e resiliência. Crie uma narrativa interna de superação e empoderamento.
> 
> **3. Analisador de Propósito Profissional:**
> 
> *   **Área de Melhoria:** Alinhar paixão e propósito com oportunidades de carreira.
> *   **Sugestões:**
>     *   **Mentoria:** Busque um mentor em sua área de interesse para obter orientação e conselhos práticos sobre como avançar em sua carreira.
>     *   **Projetos Paralelos:** Experimente diferentes áreas de interesse através de projetos paralelos ou trabalhos voluntários. Isso pode ajudá-lo a descobrir novas paixões e habilidades.
>     *   **Desenvolvimento de Habilidades:** Invista em cursos e treinamentos que aprimorem suas habilidades e o preparem para as oportunidades que você deseja.
> 
> **4. Detector de Auto-Sabotagem:**
> 
> *   **Área de Melhoria:** Transformar padrões de auto-sabotagem em comportamentos capacitadores.
> *   **Sugestões:**
>     *   **Terapia Cognitivo-Comportamental (TCC):** A TCC pode ajudá-lo a identificar e modificar padrões de pensamento e comportamento negativos que contribuem para a auto-sabotagem.
>     *   **Afirmações Positivas:** Crie afirmações positivas que desafiem suas crenças limitantes e reforcem sua autoconfiança. Repita essas afirmações diariamente.
>     *   **Defina Pequenas Vitórias:** Concentre-se em alcançar pequenas vitórias diárias para construir momentum e aumentar sua autoconfiança. Celebre cada sucesso, por menor que seja.
> 
> **5. Mapa de Gatilhos Emocionais:**
> 
> *   **Área de Melhoria:** Desenvolver inteligência emocional para lidar com gatilhos de forma construtiva.
> *   **Sugestões:**
>     *   **Treinamento em Inteligência Emocional:** Participe de workshops ou cursos de inteligência emocional para aprender a reconhecer, compreender e gerenciar suas emoções de forma eficaz.
>     *   **Prática da Empatia:** Esforce-se para entender as perspectivas e emoções dos outros. Isso pode ajudá-lo a responder de forma mais compassiva e construtiva em situações desafiadoras.
>     *   **Técnicas de Relaxamento:** Experimente diferentes técnicas de relaxamento, como ioga, tai chi ou massagem, para reduzir o estresse e a ansiedade e aumentar sua resiliência emocional.
> 
> **6. Escaneamento de Energia nos Relacionamentos:**
> 
> *   **Área de Melhoria:** Aprofundar a conexão e a intimidade nos relacionamentos.
> *   **Sugestões:**
>     *   **Comunicação Não Violenta (CNV):** Aprenda a se comunicar de forma clara, honesta e respeitosa, expressando suas necessidades e sentimentos sem culpar ou criticar os outros.
>     *   **Tempo de Qualidade:** Dedique tempo de qualidade aos seus relacionamentos, concentrando-se em atividades que promovam a conexão e a intimidade. Desligue as distrações e esteja presente no momento.
>     *   **Feedback Construtivo:** Peça feedback aos seus parceiros sobre como você pode melhorar seus relacionamentos. Esteja aberto a críticas e disposto a fazer mudanças.
> 
> Lembre-se que o desenvolvimento pessoal é uma jornada contínua. Seja paciente consigo mesmo, celebre seus progressos e esteja aberto a aprender e crescer ao longo do caminho.


--------------------------------------------------------------

--- 📝 Executando o Agente 3 (Buscador de Pessoas de Sucesso) ---


--- ✅ Resultado do Agente 3 (Buscador de Pessoas de Sucesso) ---



Nome,Profissão,Sucesso,Site da Informação
Mônica Martelli,Atriz,"Reconhecida atriz brasileira, com destaque no teatro, cinema e televisão.",[Área VIP](https://areavip.com.br/famosos/confira-os-aniversariantes-famosos-do-dia-14-de-maio/)
Mariana Godoy,Jornalista,"Jornalista de renome, com passagens por importantes veículos de comunicação do Brasil.",[Área VIP](https://areavip.com.br/famosos/confira-os-aniversariantes-famosos-do-dia-14-de-maio/)
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado
Não Encontrado,Não Encontrado,Não Encontrado,Não Encontrado


--------------------------------------------------------------

--- 📝 Executando o Agente 4 (Gerador de Relatório Final) ---


--- ✅ Resultado do Agente 4 (Gerador de Relatório Final) ---



> ## Relatório Final: Uma Jornada de Autodescoberta e Crescimento
> 
> Este relatório consolida as análises detalhadas fornecidas pelos agentes, oferecendo uma visão abrangente de seus pontos fortes, áreas de melhoria e potencial para o sucesso. As informações a seguir são projetadas para inspirar e motivar você em sua jornada de autodescoberta e desenvolvimento pessoal.
> 
> ### Resumo da Análise da Personalidade
> 
> Sua data de nascimento revela uma personalidade rica e multifacetada. Você possui uma combinação única de inteligência, praticidade, introspecção e adaptabilidade. Seus pontos fortes naturais incluem uma excelente capacidade de comunicação, determinação e uma busca constante por conhecimento. A influência de Touro e Libra em sua personalidade confere-lhe um desejo de harmonia, estabilidade e um apreço pela beleza.
> 
> ### Áreas de Melhoria e Sugestões Práticas
> 
> O Agente 2 identificou áreas-chave para o crescimento pessoal e ofereceu sugestões práticas para cada uma delas:
> 
> *   **Equilibrar Introspecção e Ação:** Transforme seus insights em metas tangíveis e construa uma rede de contatos estratégica.
> *   **Resignificar Experiências Passadas:** Explore suas memórias de infância com um profissional ou através da escrita terapêutica para fortalecer sua resiliência.
> *   **Alinhar Paixão e Propósito:** Busque mentoria e experimente diferentes áreas de interesse através de projetos paralelos.
> *   **Transformar Auto-Sabotagem:** Utilize a Terapia Cognitivo-Comportamental (TCC) e afirmações positivas para desafiar crenças limitantes.
> *   **Desenvolver Inteligência Emocional:** Participe de workshops e pratique a empatia para lidar com gatilhos emocionais de forma construtiva.
> *   **Aprofundar a Conexão nos Relacionamentos:** Aprenda a Comunicação Não Violenta (CNV) e dedique tempo de qualidade aos seus relacionamentos.
> 
> ### Inspiração de Pessoas de Sucesso
> 
> Embora a pesquisa tenha retornado um número limitado de personalidades com a mesma data de nascimento, encontramos exemplos inspiradores de sucesso:
> 
> *   **Mônica Martelli:** Atriz brasileira reconhecida por seu talento no teatro, cinema e televisão.
> *   **Mariana Godoy:** Jornalista de renome com uma carreira sólida em importantes veículos de comunicação do Brasil.
> 
> Esses exemplos demonstram que é possível alcançar o sucesso em diversas áreas, utilizando seus talentos e habilidades únicas.
> 
> ### Mensagem de Incentivo
> 
> A jornada de autodescoberta é um processo contínuo e transformador. Cada desafio superado, cada pequena vitória celebrada, o aproxima de seu pleno potencial. Lembre-se de que você possui uma combinação única de talentos e habilidades que o tornam especial e capaz de realizar grandes feitos.
> 
> **Acredite em si mesmo, abrace suas qualidades e trabalhe com dedicação para superar seus desafios. O mundo precisa do seu talento, da sua paixão e da sua visão única. Vá em frente e conquiste seus sonhos!**
> 


--------------------------------------------------------------
